# Import Libraries

In [24]:
# Pandas & Numpy 
import pandas as pd 
import numpy as np 

# Suprise & Recommenders 
import surprise 
from recommenders.models.surprise.surprise_utils import compute_ranking_predictions, predict
from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k

# Reading Data

In [25]:
# Read Train & Test Data
train = pd.read_csv("../../00_Data/online_retail_ratings_train.csv", index_col=0)
test = pd.read_csv("../../00_Data/online_retail_ratings_test.csv", index_col=0)

In [26]:
# Check Shapes
print("Shape of Train:", train.shape)
print("Shape of Test: \t", test.shape)

Shape of Train: (349978, 3)
Shape of Test: 	 (87558, 3)


# Preparation of Train Dataset for Surprise Models



In [29]:
# Reader 
reader = surprise.Reader(rating_scale=(-2,2))

# Build Train Set from Custom Dataset
train_set = surprise.Dataset.load_from_df(train[['CustomerID', 'StockCode', 'purchased']], reader=reader).build_full_trainset()

# Baseline Model

In [30]:
# Delete Model first 
try:
    del(basemodel)
except:
    pass

# Create Object for Model
basemodel = surprise.NMF(random_state=0, verbose=False, n_factors=10, n_epochs=50, reg_pu=0.1, reg_qi=0.1)

# Fit Model
basemodel.fit(train_set)

# Prediction: Test Set only

In [31]:
# Predict Test Only
test_pred = predict(basemodel, test, usercol='CustomerID', itemcol='StockCode')

In [32]:
# Check Range of predicions
test_pred.prediction.describe()

count    87558.000000
mean         0.058892
std          0.541396
min         -2.000000
25%         -0.026785
50%          0.001746
75%          0.132778
max          2.000000
Name: prediction, dtype: float64

In [ ]:
# Convert Predictions
test_pred['prediction'] = np.where((test_pred['prediction']>0),1,0)

# Convert Test
test['purchased'] = np.where((test['purchased']>0),1,0)

In [ ]:
# Check Distribution
test_pred['prediction'].value_counts()

In [ ]:
# Sort Index of both Datasets to use Accuracy Score 
test = test.sort_values(by=['CustomerID', 'StockCode'])
test_pred = test_pred.sort_values(by=['CustomerID', 'StockCode'])

# Reset indeces for both DataFrames
test = test.reset_index(drop=True)
test_pred = test_pred.reset_index(drop=True)

In [ ]:
# First Reorder columns for test 
test = test[['CustomerID', 'StockCode', 'purchased']]
# Head of Test
test.head()

In [ ]:
# Head of Test_pred
test_pred.head()

In [ ]:
# Import Accuracy 
from sklearn.metrics import accuracy_score

# Accuracy 
accuracy_score(test.purchased, test_pred.prediction)

# Prediction: Top N

In [33]:
# Predict All Pairs of Users & Items that are NOT in the Trainset 
predictions = compute_ranking_predictions(basemodel, train, usercol='CustomerID', itemcol='StockCode', remove_seen=True)

In [34]:
# Write function to filter out top N 
def filter_top_n(predictions: pd.DataFrame, n: int) -> pd.DataFrame:
    
    # Group the dataframe by 'CustomerID', and for each group, sort by 'prediction' in descending order, then take the top N rows
    top_n_per_customer = predictions.groupby('CustomerID', group_keys=False).apply(lambda group: group.sort_values('prediction', ascending=False).head(n))
    
    return top_n_per_customer

# Filter Top 10 
top_10 = filter_top_n(predictions, 10)

# Filter Top 20 
top_20 = filter_top_n(predictions, 20)

In [36]:
# Evaluate Precision at 10 
eval_precision_10 = precision_at_k(test, top_10, col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=10)
print('precision at 10 \t:', eval_precision_10)

# Evaluate Recall at 10 
eval_recall_10 = recall_at_k(test, top_10,col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=10)
print('recall at 10 \t:', eval_recall_10)

recall at 10 	: 0.0017577644496014988


In [ ]:
# Evaluate Precision at 20 
eval_precision_20 = precision_at_k(test, top_20, col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=20)
print('precision at 20 \t:', eval_precision_20)

# Evaluate Recall at 20 
eval_recall_20 = recall_at_k(test, top_20,col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=20)
print('recall at 20 \t:', eval_recall_20)

# Hyperparameter Tuning

In [127]:
# Define Set of Hyperparameters 
n_factors = [20, 2000]
reg_bu = [0.01]
reg_bi = [0.01]
lr_bu = [0.001]
lr_bi = [0.001]

# Initiate Lists 
factor_nr = []
regulation_user = []
regulation_item = []
lr_user = []
lr_item = []
recall_10 = []

for factors in n_factors:
    for reg_user in reg_bu:
        for reg_item in reg_bi:
            for learning_user in lr_bu:
                for learning_item in lr_bi:
                    try:
                        del(model)
                    except:
                        pass
                    
                    # Define Model 
                    model = surprise.NMF(random_state=0,
                                        verbose=False,
                                        biased=False,
                                        n_factors=factors,
                                        n_epochs=20,
                                        reg_bu=reg_user,
                                        reg_bi=reg_item,
                                        lr_bu=learning_user,
                                        lr_bi=learning_item)
                    
                    # Fit model
                    model.fit(train_set)
                    
                    # Predict All Pairs of Users & Items that are NOT in the Trainset 
                    predictions = compute_ranking_predictions(basemodel, train, usercol='CustomerID', itemcol='StockCode', remove_seen=True)
                    
                    # Filter Top 10 
                    top_10 = filter_top_n(predictions, 10)
                    
                    # Evaluate Recall at 10 
                    eval_recall_10 = recall_at_k(test, top_10,col_user="CustomerID", col_item="StockCode",
                                                col_rating="purchased", col_prediction="prediction", 
                                                relevancy_method="top_k", k=10)

                    print("recall at 10 for", factors, "factors: \t", eval_recall_10)
                    factor_nr.append(factors)
                    regulation_user.append(reg_user)
                    regulation_item.append(reg_item)
                    lr_user.append(learning_user)
                    lr_item.append(learning_item)
                    recall_10.append(eval_recall_10)


recall at for 20 factors: 	 0.003002190400353022
recall at for 2000 factors: 	 0.003002190400353022


In [123]:
dict_param = {
    "factor": factor_nr,
    "regulation_user": regulation_user,
    "regulation_item": regulation_item,
    "learning_rate_user": lr_user,
    "learning_rate_item": lr_item,
    "recall_10": recall_10
}

In [124]:
df_param = pd.DataFrame(dict_param)
df_param.head()

,factor,recall_10
0,20,0.00
1,200,0.00


In [125]:
df_param.describe()

,factor,recall_10
count,2.00,2.00
mean,110.00,0.00
std,127.28,0.00
min,20.00,0.00
25%,65.00,0.00
50%,110.00,0.00
75%,155.00,0.00
max,200.00,0.00
